<a href="https://colab.research.google.com/github/d9249/DACON/blob/main/Baseline_DenseNet169_BS64EP100L152-2.48872.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Oct  5 00:35:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
    from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
from glob import glob

from PIL import Image
import cv2
from tqdm import tqdm

import os
import shutil
import json

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')

In [4]:
!mkdir open_data
!unzip -uq "/content/drive/MyDrive/DACON_2021_recognize_traffic_signal_motion/open.zip" -d "/content/open_data/"

mkdir: cannot create directory ‘open_data’: File exists


In [5]:
data_path = '/content/open_data/open'
train_path = data_path + '/train'
test_path = data_path + '/test'
hand_gesture = pd.read_csv(data_path + '/hand_gesture_pose.csv')
sample_submission = pd.read_csv(data_path + '/sample_submission.csv')

In [6]:
train_folders = sorted(glob(train_path + '/*'), key = lambda x : int(x.split('/')[-1]))
test_folders  = sorted(glob(test_path + '/*'), key = lambda x : int(x.split('/')[-1]))

In [7]:
train_folder = train_folders[0]

image_paths = sorted(glob(train_folder + '/*.png'), key = lambda x : int(x.split('/')[-1].replace('.png','')))
json_path   = glob(train_folder + '/*.json')[0]

In [8]:
answers = []
for train_folder in train_folders :
    json_path = glob(train_folder + '/*.json')[0]
    js = json.load(open(json_path))
    cat = js.get('annotations')[0].get('category_id')
    cat_name = js.get('annotations')[0].get('category_name')
    answers.append([train_folder.replace(data_path,''),cat, cat_name])

answers = pd.DataFrame(answers, columns = ['train_path','answer', 'answer_name'])

In [9]:
classes = pd.get_dummies(answers[['answer']], columns = ['answer']).to_numpy()

In [10]:
np.random.shuffle(train_folders) # 일반화 가능성 높이기

In [11]:
from keras.preprocessing import image

images  = []
targets = []
for train_folder in tqdm(train_folders) :
    image_paths = sorted(glob(train_folder + '/*.png'), key = lambda x : int(x.split('/')[-1].replace('.png','')))
    query_path  = train_folder.replace(data_path,'')
    target = classes[int(train_folder.split('/')[-1])]
    for image_path in image_paths:
        img = image.load_img(image_path, target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img/255
        images.append(img)
        targets.append(target)

100%|██████████| 649/649 [05:02<00:00,  2.14it/s]


In [12]:
X = np.array(images)
print('Train X Shape : ', X.shape)
y = np.array(targets)
print('Train y Shape : ', y.shape)

X_train, X_valid, y_train, y_valid = train_test_split(
      X, 
      y, 
      random_state = 2021, 
      test_size = 0.2, 
      stratify = y
    )

Train X Shape :  (5887, 224, 224, 3)
Train y Shape :  (5887, 157)


In [13]:
from keras.preprocessing import image
from PIL import ImageFile, Image
ImageFile.LOAD_TRUNCATED_IMAGES = True

test_images  = []
for test_folder in tqdm(test_folders, total = len(test_folders)) :
    image_paths = sorted(glob(test_folder + '/*.png'), key = lambda x : int(x.split('/')[-1].replace('.png','')))
    query_path  = test_folder.replace(data_path,'')
    test_image = []
    for image_path in image_paths:
        img = image.load_img(image_path, target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img/255
        test_image.append(img)
    test_images.append(test_image)

test_images = np.array(test_images)
print(test_images.shape)

100%|██████████| 217/217 [01:43<00:00,  2.10it/s]

(217,)


In [14]:
baseModel = DenseNet169(weights='imagenet', include_top=False)
baseModel.trainable = False

model_in = Input(shape = (224,224,3))
base_model = baseModel(model_in)
head_model = AveragePooling2D(pool_size=(5, 5))(base_model)
head_model = Flatten(name="flatten")(head_model)
head_model = Dense(1024, activation="relu")(head_model)
head_model = Dropout(0.2)(head_model)
head_model = Dense(512, activation="relu")(head_model)
head_model = Dropout(0.2)(head_model)
head_model = Dense(256, activation="relu")(head_model)
head_model = Dropout(0.2)(head_model)
model_out = Dense(classes.shape[1], activation="softmax")(head_model)

model = Model(inputs=model_in, outputs=model_out)

In [15]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])

In [16]:
model.fit(
        X_train,
        y_train, 
        validation_data = (X_valid, y_valid),
        epochs = 100,
        verbose = 1,
        batch_size = 64
        )

Epoch 1/100
74/74 [==============================] - 39s 210ms/step - loss: 4.3492 - accuracy: 0.0769 - val_loss: 2.9302 - val_accuracy: 0.2615
Epoch 2/100
74/74 [==============================] - 11s 154ms/step - loss: 2.4441 - accuracy: 0.3221 - val_loss: 1.2968 - val_accuracy: 0.6333
Epoch 3/100
74/74 [==============================] - 11s 154ms/step - loss: 1.4055 - accuracy: 0.5617 - val_loss: 0.7345 - val_accuracy: 0.7878
Epoch 4/100
74/74 [==============================] - 11s 154ms/step - loss: 0.9142 - accuracy: 0.7035 - val_loss: 0.5177 - val_accuracy: 0.8531
Epoch 5/100
74/74 [==============================] - 11s 154ms/step - loss: 0.6338 - accuracy: 0.7955 - val_loss: 0.3489 - val_accuracy: 0.8854
Epoch 6/100
74/74 [==============================] - 11s 153ms/step - loss: 0.4528 - accuracy: 0.8492 - val_loss: 0.2409 - val_accuracy: 0.9236
Epoch 7/100
74/74 [==============================] - 11s 153ms/step - loss: 0.3410 - accuracy: 0.8938 - val_loss: 0.1767 - val_accuracy:

In [17]:
predictions = []
for test_image in tqdm(test_images, total = len(test_images)) : 
    prediction = np.mean(model.predict(np.array(test_image)), axis = 0)
    predictions.append(prediction)

sample_submission.iloc[:,1:] = predictions
sample_submission.to_csv('./Baseline_DenseNet169_BS64EP50.csv', index=False)

100%|██████████| 217/217 [00:21<00:00, 10.26it/s]


In [18]:
from google.colab import files
files.download('./Baseline_DenseNet169_BS64EP50.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>